# 加上两个用户行为特征

1. 8234 base


In [1]:
import sys
sys.path.append("..")
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline

train_df = pd.read_table('../../round1_ijcai_18_train_20180301.txt',sep=' ')
test_df = pd.read_table('../../round1_ijcai_18_test_a_20180301.txt',sep=' ')

# 线下线上数据统一进行特征处理
test_df['is_trade'] = -1
total_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

# 1 先处理时序数据

In [2]:
# 时间处理: 分离天, 星期几, 上中下午/晚上, 小时数
# date最终不使用，直接用day(第 0 - 7 天)

from sklearn import preprocessing
from datetime import datetime
le = preprocessing.LabelEncoder()

def extract_date(x):
    d = datetime.fromtimestamp(x)
    return d.strftime('%Y-%m-%d')
def extract_weekday(x):
    d = datetime.fromtimestamp(x)
    return d.weekday()
def extract_hour(x):
    d = datetime.fromtimestamp(x)
    return d.hour

total_df['date'] = total_df['context_timestamp'].apply(lambda x: extract_date(x))
total_df['day'] = le.fit_transform(total_df['date'])
total_df['weekday'] = total_df['context_timestamp'].apply(lambda x: extract_weekday(x))
total_df['hour'] = total_df['context_timestamp'].apply(lambda x: extract_hour(x))

# 2 先分离训练集, 测试集, 线上集



In [3]:
# 获取训练测试的索引, 6全集数据验证, 7为生成上线文件
import numpy as np
test_day= 6
starts = list(range(0,test_day))
ends = [test_day]
train_indices = total_df[total_df['day'].isin(starts)].index.values
test_indices = total_df[total_df['day'].isin(ends)].index.values
print(train_indices.shape, test_indices.shape)

tmp_df = total_df.copy()

# 把测试索引的label提取
y_test = tmp_df.iloc[test_indices]['is_trade']

# 把需要训练和测试的数据提取, 并且强制去掉测试集的label列
tmp_df.loc[test_indices, 'is_trade'] = np.nan

# 得出训练测试必须的数据集 ,并且添加一列data_set作为标记
train_tmp = tmp_df.iloc[train_indices].copy()
train_tmp['data_set'] = 'training'
test_tmp = tmp_df.iloc[test_indices].copy()
test_tmp['data_set'] = 'testing'
raw_df = train_tmp.append(test_tmp)


(420717,) (57421,)


### 重复列特征

In [4]:
%run ../util/time_utils.py
dup_feat = ['item_id', 'item_brand_id', 'shop_id', 'user_id']
raw_df = generateColDupByDay(raw_df, dup_feat, list(range(1, 8)), verbose=False)

### 交易率特征选择

*固定smooth为10*

1. 特征是否独立提升?
2. 平滑是否不会改变正负向?

##### 正向特征

> item_city_id, shop_id, user_gender_id, item_sales_level, item_collected_level, shop_review_num_level

##### 负向特征

> item_price_level,
item_id,item_brand_id,item_pv_level,user_age_level,
user_occupation_id,user_star_level,context_page_id,
shop_review_positive_rate,shop_star_level,
shop_score_service,shop_score_description,day,hour


In [5]:
# 计算前一天的交易率set到下一天，第0天用回自己
# %run ../util/time_utils.py

# setTradeRateByDate(raw_df, ['item_city_id', 'item_id', 'item_brand_id', 'shop_id', 'user_id'])

# 计算前一天的交易率set到下一天，第0天用回自己
%run ../util/trade_info.py

# trade_rela = ['item_city_id', 'shop_id', 'user_gender_id', 'item_sales_level', 'item_collected_level', 'shop_review_num_level']
trade_rela = ['item_city_id', 'item_id', 'item_brand_id', 'shop_id', 'user_id']

# colSm = {}
# for col in trade_rela:
#     colSm[col] = [15*(mean0) , 15]
generateTradeRateByDate(raw_df, trade_rela, 7, None, verbose=False, glbSmoothing=200, glbMean0=0.05)
print(raw_df.shape)

C:\Anaconda3\lib\site-packages\pandas\core\series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


(478138, 71)


### 复合类型拆解

In [6]:
%run ../util/complex_type.py
from sklearn import preprocessing
from datetime import datetime
le = preprocessing.LabelEncoder()
raw_df = process_complex_types(raw_df, get_icl_map(raw_df), get_ipl_map(raw_df))

raw_df['item_category_1'] = le.fit_transform(raw_df.item_category_1)
raw_df['item_category_2'] = le.fit_transform(raw_df.item_category_2)

get_icl_map ... 
get_ipl_map ... 
processing predict_category_property ...
processing item_property_list ...
processing item_category_list ...
generating item_category_1, item_category_2 ...


In [7]:
%run ../util/complex_type.py

cnt_user_item_review = raw_df[["user_id", "item_id", "instance_id"]].groupby(["user_id", "item_id"])['instance_id'].count().to_dict() 
cnt_user_cate_review = raw_df[["user_id", "item_category_1", "instance_id"]].groupby(["user_id", "item_category_1"])['instance_id'].count().to_dict()  

f1 = set_review_cnt("user_id", "item_id", cnt_user_item_review)
f2 = set_review_cnt("user_id", "item_category_1", cnt_user_cate_review)

tmp = raw_df.sort_values(by="context_timestamp")
tmp["item_review_cnt"] = tmp[["user_id", "item_id"]].apply(f1, axis=1)
tmp["cate_review_cnt"] = tmp[["user_id", "item_category_1"]].apply(f2, axis=1)
raw_df = tmp.sort_index()


# 3 生成矩阵数据

In [8]:
train_df = raw_df[raw_df['data_set'] == 'training']
test_df = raw_df[raw_df['data_set'] == 'testing']

non_feat_columns = ['data_set', 'context_timestamp', 'instance_id', 'is_trade', 'context_id',
                   'item_property_list', 'item_category_list', 'date', 'predict_category_property',
                    'predict_richness', 'predict_category_property', 'item_property_richness', 'item_property_list', 
                    'item_category_list', 'item_category_1', 'item_category_2'
                   ]

D = train_df[['date']]
X_train = train_df.drop(non_feat_columns, axis=1)
y_train = train_df[['is_trade']].values.ravel()
X_test = test_df.drop(non_feat_columns, axis=1)
# y_test is already exists

# X_online = test_df.drop(non_feat_columns, axis=1).values
print(D.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(420717, 1) (420717, 64) (420717,) (57421, 64) (57421,)


In [ ]:
# 训练模型
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import xgboost

#import lightgbm as lgb

%run ../util/time_series_split.py

# clf = lgb.LGBMClassifier(n_jobs=20)
clf = xgboost.XGBClassifier(n_jobs=7,max_depth=5,n_estimators=100,min_child_weight=5, scale_pos_weight=1)
# clf = xgboost.XGBClassifier(n_jobs=7, max_depth=3, n_estimators=230)

m = None

# 如果移动到线上集, 则输出模型
if sum(y_test == -1) > 0:
    m = clf.fit(X_train, y_train)
    result = pd.DataFrame()
    result['instance_id'] = test_df['instance_id']
    result['predicted_score'] = pd.DataFrame(m.predict_proba(X_test))[1].values
    result.to_csv('submits/4_7348_8100_a8172_b8069.csv', sep = ' ', header=True, index = False)
else:
    # 分离a,b榜
    X_val_a, X_val_b, y_val_a, y_val_b = train_test_split(X_test, y_test, test_size=0.7, shuffle=True, random_state=6)
    m = clf.fit(X_train, y_train)
    
    val_a_loss = log_loss(y_val_a, m.predict_proba(X_val_a))
    val_b_loss = log_loss(y_val_b, m.predict_proba(X_val_b))
    print('(%s -> %s) train logloss: %.5f, test logloss: %.5f, a: %.5f, b: %.5f' % \
          (starts, ends, \
           log_loss(y_train, m.predict_proba(X_train)), \
           log_loss(y_test, m.predict_proba(X_test)),\
          val_a_loss, val_b_loss))
    
# ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07454, test logloss: 0.08186, a: 0.08258, b: 0.08156


([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08162, b: 0.08064


# 4 模型分析

In [ ]:
for i in range(1000):
    # 分离a,b榜
    X_val_a, X_val_b, y_val_a, y_val_b = train_test_split(X_test, y_test, test_size=0.7, shuffle=True, random_state=i)
    
    val_a_loss = log_loss(y_val_a, m.predict_proba(X_val_a))
    val_b_loss = log_loss(y_val_b, m.predict_proba(X_val_b))
    print('%d (%s -> %s) train logloss: %.5f, test logloss: %.5f, a: %.5f, b: %.5f' % \
          (i, starts, ends, \
           log_loss(y_train, m.predict_proba(X_train)), \
           log_loss(y_test, m.predict_proba(X_test)),\
          val_a_loss, val_b_loss))

0 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08480, b: 0.07927
1 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07698, b: 0.08263
2 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08372, b: 0.07974
3 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08042, b: 0.08115
4 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07785, b: 0.08226
5 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08439, b: 0.07945
6 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08162, b: 0.08064
7 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08499, b: 0.07919
8 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07787, b: 0.08224
9 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08530, b: 0.07906


82 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08022, b: 0.08124
83 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08384, b: 0.07969
84 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08008, b: 0.08130
85 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08002, b: 0.08132
86 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07667, b: 0.08276
87 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08498, b: 0.07920
88 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08383, b: 0.07969
89 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07763, b: 0.08235
90 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08080, b: 0.08099
91 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07722, b

163 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07560, b: 0.08322
164 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08392, b: 0.07965
165 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08105, b: 0.08088
166 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07917, b: 0.08169
167 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08252, b: 0.08025
168 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08181, b: 0.08056
169 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08110, b: 0.08086
170 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07945, b: 0.08157
171 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08390, b: 0.07966
172 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 

244 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08145, b: 0.08071
245 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07734, b: 0.08247
246 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08264, b: 0.08020
247 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07938, b: 0.08160
248 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07844, b: 0.08200
249 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08626, b: 0.07865
250 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08075, b: 0.08101
251 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08059, b: 0.08108
252 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08327, b: 0.07993
253 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 

325 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07663, b: 0.08277
326 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08385, b: 0.07968
327 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08282, b: 0.08012
328 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07931, b: 0.08163
329 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08128, b: 0.08078
330 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07939, b: 0.08159
331 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08230, b: 0.08035
332 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08216, b: 0.08041
333 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07914, b: 0.08170
334 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 

...

487 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08037, b: 0.08117
488 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07966, b: 0.08148
489 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08105, b: 0.08088
490 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07966, b: 0.08148
491 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08182, b: 0.08055
492 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08122, b: 0.08081
493 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08307, b: 0.08002
494 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07861, b: 0.08193
495 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07853, b: 0.08196
496 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 

568 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07848, b: 0.08199
569 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08202, b: 0.08047
570 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08200, b: 0.08048
571 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07998, b: 0.08134
572 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07402, b: 0.08389
573 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08299, b: 0.08005
574 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08554, b: 0.07896
575 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08433, b: 0.07948
576 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07607, b: 0.08302
577 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 

649 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07941, b: 0.08158
650 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08650, b: 0.07855
651 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07856, b: 0.08195
652 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07953, b: 0.08153
653 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07939, b: 0.08159
654 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08175, b: 0.08058
655 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08240, b: 0.08030
656 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07702, b: 0.08261
657 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08510, b: 0.07915
658 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 

730 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07362, b: 0.08407
731 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07797, b: 0.08220
732 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07878, b: 0.08185
733 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08304, b: 0.08003
734 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07901, b: 0.08176
735 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07910, b: 0.08172
736 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07817, b: 0.08212
737 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08255, b: 0.08024
738 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07507, b: 0.08345
739 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 

811 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08247, b: 0.08027
812 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07451, b: 0.08369
813 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08175, b: 0.08058
814 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07822, b: 0.08209
815 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07731, b: 0.08249
816 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07855, b: 0.08195
817 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.07991, b: 0.08137
818 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08512, b: 0.07914
819 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08245, b: 0.08028
820 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 

In [11]:
# print(X_val_a.shape, y_val_a.shape, X_val_b.shape, y_val_b.shape)

# rand_states = []
# bias = 0.0
# for i in range(0, 3000):
#     X_val_a, X_val_b, y_val_a, y_val_b = train_test_split(X_test, y_test, test_size=0.7, shuffle=True, random_state=i)
#     # train_loss = log_loss(y_train, m.predict_proba(X_train))
    
#     val_a_loss = log_loss(y_val_a, m.predict_proba(X_val_a))
#     val_b_loss = log_loss(y_val_b, m.predict_proba(X_val_b))
#     bias = bias + val_a_loss - val_b_loss
#     if val_a_loss > 0.089:
#         rand_states.append(i)
    
#     print('第 %s 次: A: %.5f, B: %.5f, A_B_mean: %.5f, bias: %.5f' % \
#           ( i, val_a_loss, val_b_loss, (val_a_loss*0.3+val_b_loss*0.7), bias))
    
# test_loss = log_loss(y_test, m.predict_proba(X_test))
# print('test logloss: %.5f, 总的bias: %s' % (test_loss, bias))
# print(rand_states)